In [ ]:
from chi import server, context, lease
import os

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@UC")

In [ ]:
l = lease.get_lease(f"node_project29_gpu_p100") 
l.show()

In [ ]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node_project29_gpu_p100", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

In [ ]:
s.associate_floating_ip()

In [ ]:
s.refresh()
s.check_connectivity()

In [ ]:
s.refresh()
s.show(type="widget")

In [ ]:
s.execute("git clone https://github.com/sid150/ChXpert-Chest-X-rays-Prediction.git")

In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

In [ ]:
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\") then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json")
s.execute("sudo systemctl restart docker")

In [ ]:
s.execute("sudo apt update")
s.execute("sudo apt -y install nvtop")

ssh into node and run these


curl https://rclone.org/install.sh | sudo bash



sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf



mkdir -p ~/.config/rclone
nano  ~/.config/rclone/rclone.conf




[chi_tacc]
type = swift
user_id = 1d516491f6c35cc485ec339f5bd04b3b60ad982213c9804cdab35e13dd10f04b
application_credential_id = db008fab60234a0081d830d48694b064
application_credential_secret = KgMwl26eF2zr6N0PMWDhfBm1FOTB6k-DNACTPOrb5EXxXPEYKu1EpH9sZhfEJDX96QUNOVHV5-Dbv9zZgXR4Lw
auth = https://chi.tacc.chameleoncloud.org:5000/v3
region = CHI@TACC




rclone lsd chi_tacc:




sudo mkdir -p /mnt/object
sudo chown -R cc /mnt/object
sudo chgrp -R cc /mnt/object


replace object name


rclone mount chi_tacc:object-persist-project29 /mnt/object --read-only --allow-other --vfs-cache-mode=full   --dir-cache-time=72h --swift-fetch-until-empty-page --daemon

ls /mnt/object

### run in persisitent cpu instance

HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4 ) MINIO_ROOT_PASSWORD=myminioadmin123 AWS_ACCESS_KEY_ID=myminioadmin AWS_SECRET_ACCESS_KEY=myminioadmin123 POSTGRES_USER=mlflowuser POSTGRES_PASSWORD=mlflowpass POSTGRES_DB=mlflowdb docker compose -f docker-storage.yaml up -d

### run in bare metal gpu instance

HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4 ) MINIO_ROOT_USER=myminioadmin MINIO_ROOT_PASSWORD=myminioadmin123 AWS_ACCESS_KEY_ID=myminioadmin AWS_SECRET_ACCESS_KEY=myminioadmin123 POSTGRES_USER=mlflowuser POSTGRES_PASSWORD=mlflowpass POSTGRES_DB=mlflowdb docker compose -f  ChXpert-Chest-X-rays-Prediction/docker/docker-compose-ray-cuda-req.yaml up -d

``` bash
# run on node-mltrain
docker build -t jupyter-ray -f ChXpert-Chest-X-rays-Prediction/docker/Dockerfile.jupyter-ray .
```

Run

``` bash
# run on node-mltrain
HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4)

docker run -d --rm -p 8888:8888 \
  -v ~/ChXpert-Chest-X-rays-Prediction/:/home/jovyan/work/ \
  --mount type=bind,source=/mnt/object,target=/mnt/data \
  -e RAY_ADDRESS=http://${HOST_IP}:8265/ \
  --name jupyter \
  jupyter-ray
```

Then, run

``` bash
# run on node-mltrain
docker logs jupyter
```

HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4 ) MINIO_ROOT_USER=myminioadmin MINIO_ROOT_PASSWORD=myminioadmin123 AWS_ACCESS_KEY_ID=myminioadmin AWS_SECRET_ACCESS_KEY=myminioadmin123 POSTGRES_USER=mlflowuser POSTGRES_PASSWORD=mlflowpass POSTGRES_DB=mlflowdb docker compose -f  ChXpert-Chest-X-rays-Prediction/docker/docker-compose-ray-cuda-req.yaml down

## run in bare metal instance local 

HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4 ) MINIO_ROOT_USER=myminioadmin MINIO_ROOT_PASSWORD=myminioadmin123 AWS_ACCESS_KEY_ID=myminioadmin AWS_SECRET_ACCESS_KEY=myminioadmin123 POSTGRES_USER=mlflowuser POSTGRES_PASSWORD=mlflowpass POSTGRES_DB=mlflowdb docker compose -f  ChXpert-Chest-X-rays-Prediction/docker/docker-compose-ray-cuda-local.yaml up -d

HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4 ) MINIO_ROOT_USER=myminioadmin MINIO_ROOT_PASSWORD=myminioadmin123 AWS_ACCESS_KEY_ID=myminioadmin AWS_SECRET_ACCESS_KEY=myminioadmin123 POSTGRES_USER=mlflowuser POSTGRES_PASSWORD=mlflowpass POSTGRES_DB=mlflowdb docker compose -f  ChXpert-Chest-X-rays-Prediction/docker/docker-compose-ray-cuda-local.yaml down

``` bash
# run on node-mltrain
docker build -t jupyter-ray -f ChXpert-Chest-X-rays-Prediction/docker/Dockerfile.jupyter-ray .
```

Run

``` bash
# run on node-mltrain
HOST_IP=$(curl --silent http://169.254.169.254/latest/meta-data/public-ipv4)

docker run -d --rm -p 8888:8888 \
  -v ~/ChXpert-Chest-X-rays-Prediction/:/home/jovyan/work/ \
  --mount type=bind,source=/home/cc/data,target=/mnt/data \
  -e RAY_ADDRESS=http://${HOST_IP}:8265/ \
  --name jupyter \
  jupyter-ray
```

Then, run

``` bash
# run on node-mltrain
docker logs jupyter
```